<a href="https://colab.research.google.com/github/mldurga/projects/blob/master/distilbert_transformers_te.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers datasets seqeval -Uqq

     |████████████████████████████████| 3.4 MB 5.4 MB/s 
     |████████████████████████████████| 306 kB 42.9 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 596 kB 44.5 MB/s 
     |████████████████████████████████| 895 kB 42.4 MB/s 
     |████████████████████████████████| 61 kB 427 kB/s 
     |████████████████████████████████| 3.3 MB 36.5 MB/s 
     |████████████████████████████████| 1.1 MB 38.8 MB/s 
     |████████████████████████████████| 132 kB 42.3 MB/s 
     |████████████████████████████████| 243 kB 45.4 MB/s 
     |████████████████████████████████| 192 kB 44.1 MB/s 
     |████████████████████████████████| 271 kB 42.7 MB/s 
     |████████████████████████████████| 160 kB 44.3 MB/s 


In [3]:
!pip install accelerate -qq
!apt install git-lfs

     |████████████████████████████████| 58 kB 2.9 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 1s (1,806 kB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.3.4-1_amd64.deb ...
Unpacking git-lfs (2.3.4-1) ...
Setting up git-lfs (2.3.4-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [4]:
from huggingface_hub import notebook_login

In [5]:
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [6]:
from datasets import load_dataset

In [7]:
raw_data=load_dataset("wikiann",'te')

Downloading:   0%|          | 0.00/3.94k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/12.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/234M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset wikiann downloaded and prepared to /root/.cache/huggingface/datasets/wikiann/te/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
raw_data

DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 1000
    })
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 1000
    })
})

In [9]:
raw_data['train'][0]['ner_tags']

[5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0]

In [10]:
label_names=raw_data['train'].features['ner_tags'].feature.names

In [11]:
labels=raw_data['train'][0]['ner_tags']

In [12]:
from transformers import AutoTokenizer

In [13]:
tokenizer=AutoTokenizer.from_pretrained('neuralspace-reverie/indic-transformers-te-distilbert')
tokenizer.is_fast

Downloading:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/429k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

True

In [14]:
inputs=tokenizer(raw_data['train'][0]['tokens'], is_split_into_words=True)

In [15]:
inputs.tokens()

['[CLS]',
 'ఆంధర',
 'పరదశల',
 ',',
 'వరు',
 'జనభల',
 '2',
 '%',
 'కంట',
 'తకకువగ',
 '##ను',
 ',',
 'తమళనడుల',
 'వరు',
 '3',
 '%',
 'కనన',
 'తకకువ',
 'మంద',
 'ఉననరు',
 '.',
 '[SEP]']

In [16]:
len(inputs.word_ids())

22

In [17]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [18]:
align_labels_with_tokens(labels,inputs.word_ids())

[-100, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, -100]

In [19]:
labels

[5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0]

In [20]:
def tokenize_and_align_labels(examples):
  tokenized_inputs=tokenizer(examples['tokens'],truncation=True,is_split_into_words=True)
  all_labels=examples['ner_tags']
  new_labels=[]
  for i, labels in enumerate(all_labels):
    new_labels.append(align_labels_with_tokens(labels,tokenized_inputs.word_ids(i)))
  tokenized_inputs['labels']=new_labels
  return tokenized_inputs

In [21]:
tokenized_datasets=raw_data.map(tokenize_and_align_labels,batched=True,remove_columns=raw_data['train'].column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [22]:
tokenized_datasets['train'][4]['labels']

[-100, 3, 4, 0, 3, 4, 0, -100]

In [23]:
from transformers import DataCollatorForTokenClassification

In [24]:
data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer)

In [25]:
batch=data_collator([tokenized_datasets['train'][i] for i in range(2)])
batch['labels']

tensor([[-100,    5,    6,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            5,    0,    0,    0,    0,    0,    0,    0,    0, -100],
        [-100,    5,    6,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0, -100, -100, -100, -100, -100, -100, -100, -100]])

In [26]:
from datasets import load_metric

In [27]:
metric=load_metric('seqeval')

Downloading:   0%|          | 0.00/2.48k [00:00<?, ?B/s]

In [28]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [29]:
id2label={str(i): label for i,label in enumerate(label_names)}
id2label

{'0': 'O',
 '1': 'B-PER',
 '2': 'I-PER',
 '3': 'B-ORG',
 '4': 'I-ORG',
 '5': 'B-LOC',
 '6': 'I-LOC'}

In [30]:
label2id={v:k for k,v in id2label.items()}
label2id

{'B-LOC': '5',
 'B-ORG': '3',
 'B-PER': '1',
 'I-LOC': '6',
 'I-ORG': '4',
 'I-PER': '2',
 'O': '0'}

In [31]:
from transformers import AutoModelForTokenClassification

In [32]:
model=AutoModelForTokenClassification.from_pretrained('neuralspace-reverie/indic-transformers-te-distilbert',id2label=id2label,label2id=label2id)

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at neuralspace-reverie/indic-transformers-te-distilbert were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at neuralspace-reverie/indic-transformers-te-distilbert and are newly initialized: ['classifi

In [33]:
model.config.num_labels

7

In [34]:
from transformers import TrainingArguments

In [35]:
args=TrainingArguments('neuralspace-reverie/indic-transformers-te-distilbert',
                       evaluation_strategy='epoch',
                       save_strategy='epoch',
                       learning_rate=2e-5,
                       num_train_epochs=3,
                       weight_decay=0.01,
                       push_to_hub=True)

In [36]:
from transformers import Trainer

In [37]:
trainer=Trainer(model=model,
                args=args,
                train_dataset=tokenized_datasets['train'],
                eval_dataset=tokenized_datasets['validation'],
                data_collator=data_collator,
                compute_metrics=compute_metrics,
                tokenizer=tokenizer)
trainer.train()

Cloning https://huggingface.co/durgaamma2005/indic-transformers-te-distilbert into local empty directory.
***** Running training *****
  Num examples = 1000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 375


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.362897,0.485474,0.528726,0.506178,0.882594
2,No log,0.303161,0.544604,0.630308,0.584330,0.900205
3,No log,0.293974,0.565723,0.648626,0.604344,0.904919


***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoint to neuralspace-reverie/indic-transformers-te-distilbert/checkpoint-125
Configuration saved in neuralspace-reverie/indic-transformers-te-distilbert/checkpoint-125/config.json
Model weights saved in neuralspace-reverie/indic-transformers-te-distilbert/checkpoint-125/pytorch_model.bin
tokenizer config file saved in neuralspace-reverie/indic-transformers-te-distilbert/checkpoint-125/tokenizer_config.json
Special tokens file saved in neuralspace-reverie/indic-transformers-te-distilbert/checkpoint-125/special_tokens_map.json
tokenizer config file saved in neuralspace-reverie/indic-transformers-te-distilbert/tokenizer_config.json
Special tokens file saved in neuralspace-reverie/indic-transformers-te-distilbert/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoint to neuralspace-reverie/indic-transformers-te-distilbert/checkpoint-250
Conf

TrainOutput(global_step=375, training_loss=0.37613411458333335, metrics={'train_runtime': 63.4341, 'train_samples_per_second': 47.293, 'train_steps_per_second': 5.912, 'total_flos': 17666304412944.0, 'train_loss': 0.37613411458333335, 'epoch': 3.0})

In [38]:
from transformers import pipeline

In [39]:
model_checkpoint='neuralspace-reverie/indic-transformers-te-distilbert/checkpoint-375'
token_classifier=pipeline('token-classification',model=model_checkpoint,aggregation_strategy='simple')

loading configuration file neuralspace-reverie/indic-transformers-te-distilbert/checkpoint-375/config.json
Model config DistilBertConfig {
  "_name_or_path": "neuralspace-reverie/indic-transformers-te-distilbert/checkpoint-375",
  "activation": "gelu",
  "architectures": [
    "DistilBertForTokenClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC"
  },
  "initializer_range": 0.02,
  "label2id": {
    "B-LOC": "5",
    "B-ORG": "3",
    "B-PER": "1",
    "I-LOC": "6",
    "I-ORG": "4",
    "I-PER": "2",
    "O": "0"
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "torch_dtype": "float32",
  "transformers_version": "4.15.0",
  "vocab_size": 3052

In [58]:
raw_data['test'][200]

{'langs': ['te', 'te', 'te', 'te', 'te'],
 'ner_tags': [0, 0, 5, 0, 0],
 'spans': ['LOC: చైనాలోనే'],
 'tokens': ['కానీ', 'బాల్యమంతా', 'చైనాలోనే', 'గడిపింది', '.']}

In [ ]:
'18వ శతాబ్దానికి ఇంగ్లాండులో ప్రకటనలు వారపత్రికలలో ముద్రితమయ్యేవి',
  '.'

In [43]:
token_classifier('18వ శతాబ్దానికి ఇంగ్లాండులో ప్రకటనలు వారపత్రికలలో ముద్రితమయ్యేవి')

[{'end': 25,
  'entity_group': 'LOC',
  'score': 0.5767875,
  'start': 16,
  'word': 'ఇంగలండు'}]

In [49]:
token_classifier('చెన్నై వరదలు మొత్తం నగరాన్ని నాశనం చేశాయి మరియు స్టాలిన్ వారి ప్రజలను తగినంతగా పట్టించుకోలేదు')

[{'end': 5,
  'entity_group': 'LOC',
  'score': 0.49217576,
  'start': 0,
  'word': 'చనన'},
 {'end': 55,
  'entity_group': 'LOC',
  'score': 0.59439147,
  'start': 48,
  'word': 'సటలన'}]

In [59]:
from huggingface_hub import Repository, get_full_repo_name

In [60]:
model_name='indic-transformers-te-distilbert'
repo=get_full_repo_name(model_name)
repo

'durgaamma2005/indic-transformers-te-distilbert'

In [61]:
repo.push_to_hub()

AttributeError: ignored